In [1]:
from bs4 import BeautifulSoup
from pathlib import Path
import json
import requests

In [62]:
data_dir = Path("data")

In [ ]:
for i in range(1,34):
    url = f"https://origami-database.com/models/page/{i}"
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    cards = soup.find_all("article")
    for card in cards:
        d={}
        name = card.h2.text.strip()
        inner_soup = BeautifulSoup(requests.get(card.a.attrs["href"]).text)
        details = inner_soup.select(".single-model__content__meta__item")
        for detail in details:
            property = detail.select_one(".single-model__content__meta__item__title").text.strip()
            value = detail.select_one(".single-model__content__meta__item__description").text.strip()
            property = '_'.join(property.lower().split(' '))
            if(property=="video_tutorial"):
                value = detail.a.attrs["href"]
            d[property] = value.lower()
        d["name"] = name

        slug = '_'.join(name.lower().replace('/','_').replace('\\','_').split(' '))
        image_url = card.img.attrs["src"]
        response = requests.get(image_url)

        image = f"{slug}.{image_url.split('.')[-1]}"

        if response.status_code == 200:
            with open(f"images/{image}", 'wb') as f:
                f.write(response.content)
        else:
            print(f"{name}: image could not download")

        d["image_url"] = image

        file_name = data_dir/f"{slug}.json"
        with open(file_name,"w") as fh:
            json.dump(d,fh,indent=4)